# <center>Test assignment<br>Position: Product Analyst</center>

<im src="http://vignette1.wikia.nocookie.net/cuttherope/images/c/c0/Zeptolab_logo.png/revision/latest?cb=20130529115328">

---

To complete the task you need to calculate the LTV of cohorts of users for all available days, submit the notebook with calculations and a final csv file. Please, provide comments for your code and make sure it is reproducible. 

- **Cohort** means all players who installed the game on the same day. Cohort numeration in this task starts with 3001 (for players who installed on January 1st 2017)
- **LTV** includes two sources - revenue from in-app purchases (IAP) and revenue from advertisments (Ads), which is a multiplication of total ads watched on a particular day by the corresponding eCPM, divided by 1000
- **LTV** calculation formula: 

$LTV_{\text{day N}} = \sum\limits_{i=1}^{N}{\frac{\text{Ads Revenue}_{\text{day i}} + \text{IAP Revenue}_{\text{day i}}}{\text{Cohort installs}}}$

Output file format:

Cohort_number  | $LTV_1$ | $LTV_2$ | $LTV_3$ | ... | $LTV_{N-1}$ | $LTV_N$
-------------- | ------- | ------- | ------- |-----| ----------- | ------
3001           |   0.01  |  0.015  |  0.04   | ... |     0.25    |  0.28
3002           |   0.03  |  0.04   |  0.12   | ... |     1.28    |  NaN
...            |   ...   |   ...   |   ...   | ... |      ...    |  ...
3031           |   0.02  |  0.07   |   NaN   | ... |      ...    |  NaN

In [316]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

### Load data


In [461]:
# 'installs.xlsx': contains data about when users installed the game
installs = pd.read_excel('installs.xlsx', converters={'Date': pd.to_datetime,'installs':int})
installs = installs.rename(columns={'cohort_number':'cohort'})

installs

,Date,cohort,installs
0,2017-01-01,3001,62078
1,2017-01-02,3002,62601
2,2017-01-03,3003,56958
3,2017-01-04,3004,54959
4,2017-01-05,3005,55273
5,2017-01-06,3006,59910
6,2017-01-07,3007,59922
7,2017-01-08,3008,55167
8,2017-01-09,3009,46065
9,2017-01-10,3010,45813


In [336]:
print(installs.cohort.unique())
print(installs.Date.min(),installs.Date.max())

[3001 3002 3003 3004 3005 3006 3007 3008 3009 3010 3011 3012 3013 3014
 3015 3016 3017 3018 3019 3020 3021 3022 3023 3024 3025 3026 3027 3028
 3029 3030 3031]
2017-01-01 00:00:00 2017-01-31 00:00:00


In [347]:
# 'eCPM.txt': contains the effective cost per thousand impressions (eCPM) for ads on each day
eCPM = pd.read_csv('eCPM.txt', sep='\t', converters={'Date': pd.to_datetime,'total_ads_watched_value':int})
eCPM.Date = pd.to_datetime(eCPM.Date)
# eCPM.to_csv('eCPM.csv')

eCPM

,Date,eCPM
0,2017-01-01,5.879974
1,2017-01-02,5.259028
2,2017-01-03,4.790151
3,2017-01-04,4.701444
4,2017-01-05,4.802823
...,...,...
56,2017-02-26,4.891530
57,2017-02-27,4.295929
58,2017-02-28,4.371963
59,2017-03-01,4.042482


In [304]:
print(eCPM.Date.min(), eCPM.Date.max())

2017-01-01 00:00:00 2017-03-02 00:00:00


In [378]:
# 'total_ads_watched.txt': the total number of ads watched by each cohort on each day
total_ads_watched = pd.read_csv('total_ads_watched.txt', sep='\t', converters={'Date': pd.to_datetime})
total_ads_watched = total_ads_watched.set_index('Date').sort_index(axis=1)

total_ads_watched

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,151902,2755,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-02,157271,143598,2343,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-03,95806,138308,126034,2034,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-04,68874,91469,129751,116066,1822,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-05,58871,69861,88032,125542,116459,2259,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-26,4193,4021,3982,3304,3107,4026,3902,4016,3622,4019,...,26261,30378,42447,80715,78246,1079,0,0,0,0
2017-02-27,3117,3054,3056,2426,2710,2951,3034,2926,2471,2673,...,18792,21885,25982,39798,55400,46635,841,0,0,0
2017-02-28,2878,2936,2668,2495,2322,2737,2853,2728,2633,2941,...,16413,18598,20811,29485,37173,45833,41999,785,0,0


In [379]:
total_ads_watched.sort_index(axis=1).columns.max()

'3062'

In [338]:
total_ads_watched.Date.max()

Timestamp('2017-03-02 00:00:00')

In [435]:
# 'in_game_purchases_revenue.csv': records revenue from in-game purchases for each cohort on each day
in_game_purchases = pd.read_csv('in_game_purchases_revenue.csv' )
in_game_purchases.Date = pd.to_datetime(in_game_purchases.Date)
in_game_purchases =in_game_purchases.set_index('Date')
in_game_purchases

/var/folders/ts/9c65r4gs4sqdfggp7k73h4ww0000gn/T/ipykernel_71123/995844436.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  in_game_purchases.Date = pd.to_datetime(in_game_purchases.Date)


,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,3624.0337,45.9284,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-01-02,3540.0400,3785.3424,55.7984,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-01-03,2369.7212,3301.0215,3372.0197,49.2513,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-01-04,2114.4172,2163.1421,1930.0785,2702.2086,98.4039,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-01-05,1658.5548,1782.8181,1808.8091,2812.8513,4367.9685,32.8342,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-26,414.0465,98.3052,134.4952,88.5339,144.5626,200.2294,279.1894,111.4981,348.2465,308.6020,...,289.0265,32.7355,197.0710,118.1110,88.5339,144.2336,216.7123,616.9079,512.4833,285.5720
2017-02-27,262.6078,42.6055,219.8707,174.0410,72.1497,111.5310,308.7336,101.7926,75.4726,88.5339,...,210.0336,111.6626,318.5707,200.1636,150.9781,249.4478,338.1791,462.9359,482.7088,423.7191
2017-02-28,190.5568,170.7181,292.5139,187.1023,341.6007,94.9823,275.5046,325.1507,144.2994,131.3368,...,338.2120,325.2165,298.6662,167.2307,85.3097,72.1168,357.9520,854.5775,613.9469,282.2162


In [437]:
in_game_purchases.columns.max()

'3031'

In [438]:
print(in_game_purchases.index.min(), in_game_purchases.index.max())

2017-01-01 00:00:00 2017-03-02 00:00:00


## Calculations

In [439]:
# Calculate Ads Revenue
df_ads_aligned = total_ads_watched.reindex(eCPM.index)
ads_revenue = df_ads_aligned.mul(eCPM['eCPM'], axis=0)/1000.0
ads_revenue

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,893.179766,16.199328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-02,827.092625,755.185933,12.321903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-03,458.925206,662.516204,603.721891,9.743167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-04,323.807288,430.036427,610.017125,545.677857,8.566032,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-05,282.747014,335.530042,422.802145,602.956049,559.332005,10.849578,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-26,20.510185,19.668842,19.478072,16.161615,15.197983,19.693299,19.086749,19.644384,17.717121,19.659058,...,128.456465,148.594894,207.630768,394.819832,382.742645,5.277961,0.000000,0.000000,0.000000,0.000000
2017-02-27,13.390411,13.119767,13.128359,10.421924,11.641968,12.677287,13.033849,12.569888,10.615241,11.483018,...,80.729099,94.016408,111.616829,170.969385,237.994470,200.340652,3.612876,0.000000,0.000000,0.000000
2017-02-28,12.582510,12.836084,11.664398,10.908048,10.151699,11.966063,12.473211,11.926716,11.511379,12.857944,...,71.757032,81.309772,90.984926,128.907335,162.518988,200.380190,183.618083,3.431991,0.000000,0.000000


In [467]:
# sum of ads revenue and iap revenue
common_dates = ads_revenue.index.intersection(in_game_purchases.index)
ads_revenue = ads_revenue.loc[common_dates]
in_game_purchases = in_game_purchases.loc[common_dates]

total_revenue = ads_revenue[ads_revenue.columns[:31]] + in_game_purchases
total_revenue

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,4517.213466,62.127728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-02,4367.132625,4540.528333,68.120303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-03,2828.646406,3963.537704,3975.741591,58.994467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-04,2438.224488,2593.178527,2540.095625,3247.886457,106.969932,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-05,1941.301814,2118.348142,2231.611245,3415.807349,4927.300505,43.683778,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-26,434.556685,117.974042,153.973272,104.695515,159.760583,219.922699,298.276149,131.142484,365.963621,328.261058,...,317.451180,53.553851,220.163912,139.912549,111.382236,172.100646,252.601455,658.123931,540.252515,313.033049
2017-02-27,275.998211,55.725267,232.999059,184.462924,83.791668,124.208287,321.767449,114.362488,86.087841,100.016918,...,227.518031,125.293583,333.808360,215.590281,166.327455,268.590460,360.144185,486.451816,504.111119,442.393504
2017-02-28,203.139310,183.554184,304.178298,198.010348,351.752399,106.948363,287.977811,337.077416,155.810779,144.194744,...,355.223309,338.738982,315.432679,181.675666,99.951405,90.798199,380.161573,874.159523,633.305953,299.900791


In [462]:
# increase values to 2017-03-02 date and fill by mean
installs.set_index('Date', inplace=True)

installs_df = pd.DataFrame()
date_range = pd.date_range(start='2017-01-01', end='2017-03-02')

installs_df = installs_df.reindex(date_range)
installs_df['cohort'] = installs['cohort']
installs_df['installs'] = installs['installs']

avg =installs.installs.mean()
installs_df['installs'] = installs_df['installs'].fillna(value=avg)
installs_df

,cohort,installs
2017-01-01,3001.0,62078.000000
2017-01-02,3002.0,62601.000000
2017-01-03,3003.0,56958.000000
2017-01-04,3004.0,54959.000000
2017-01-05,3005.0,55273.000000
...,...,...
2017-02-26,NaN,51367.677419
2017-02-27,NaN,51367.677419
2017-02-28,NaN,51367.677419
2017-03-01,NaN,51367.677419


In [479]:
# Calculate LTV values  
df = total_revenue.reindex(installs_df.index)
ltv = df.div(installs_df['installs'], axis=0).round(2)
ltv

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031
2017-01-01,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2017-01-02,0.07,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2017-01-03,0.05,0.07,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2017-01-04,0.04,0.05,0.05,0.06,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2017-01-05,0.04,0.04,0.04,0.06,0.09,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-26,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,...,0.01,0.00,0.00,0.0,0.0,0.00,0.00,0.01,0.01,0.01
2017-02-27,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,...,0.00,0.00,0.01,0.0,0.0,0.01,0.01,0.01,0.01,0.01
2017-02-28,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.01,0.00,0.00,...,0.01,0.01,0.01,0.0,0.0,0.00,0.01,0.02,0.01,0.01
2017-03-01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,...,0.01,0.00,0.01,0.0,0.0,0.01,0.00,0.02,0.01,0.01


In [484]:
# Transpose the DataFrame
ltv_transposed = ltv.transpose()

# Reset the index without dropping it to keep it as a DataFrame
ltv_transposed.reset_index(inplace=True)

# Rename the first column as 'Cohort_number' and adjust the others
new_columns = ['Date'] + [f'$LTV_{i+1}$' for i in range(len(ltv_transposed.columns) - 1)]
ltv_transposed.columns = new_columns

# Optionally, if the first column 'Date' should be named 'Cohort_number' and the actual values are cohorts,
# you can perform an additional step to ensure 'Cohort_number' is the identifier for cohorts.
ltv_transposed.rename(columns={'Date': 'Cohort_number'}, inplace=True)
ltv_transposed.to_csv('final.csv', index=False)

In [488]:
ltv_transposed.style.background_gradient(cmap='Greens')

,Cohort_number,$LTV_1$,$LTV_2$,$LTV_3$,$LTV_4$,$LTV_5$,$LTV_6$,$LTV_7$,$LTV_8$,$LTV_9$,$LTV_10$,$LTV_11$,$LTV_12$,$LTV_13$,$LTV_14$,$LTV_15$,$LTV_16$,$LTV_17$,$LTV_18$,$LTV_19$,$LTV_20$,$LTV_21$,$LTV_22$,$LTV_23$,$LTV_24$,$LTV_25$,$LTV_26$,$LTV_27$,$LTV_28$,$LTV_29$,$LTV_30$,$LTV_31$,$LTV_32$,$LTV_33$,$LTV_34$,$LTV_35$,$LTV_36$,$LTV_37$,$LTV_38$,$LTV_39$,$LTV_40$,$LTV_41$,$LTV_42$,$LTV_43$,$LTV_44$,$LTV_45$,$LTV_46$,$LTV_47$,$LTV_48$,$LTV_49$,$LTV_50$,$LTV_51$,$LTV_52$,$LTV_53$,$LTV_54$,$LTV_55$,$LTV_56$,$LTV_57$,$LTV_58$,$LTV_59$,$LTV_60$,$LTV_61$
0,3001,0.070000,0.070000,0.050000,0.040000,0.040000,0.020000,0.020000,0.030000,0.030000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.020000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000
1,3002,0.000000,0.070000,0.070000,0.050000,0.040000,0.040000,0.030000,0.030000,0.050000,0.020000,0.020000,0.030000,0.020000,0.020000,0.010000,0.020000,0.010000,0.010000,0.020000,0.040000,0.010000,0.010000,0.010000,0.010000,0.020000,0.020000,0.010000,0.010000,0.000000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.020000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3003,0.000000,0.000000,0.070000,0.050000,0.040000,0.040000,0.030000,0.020000,0.020000,0.010000,0.020000,0.010000,0.010000,0.010000,0.020000,0.010000,0.010000,0.010000,0.010000,0.010000,0.020000,0.010000,0.010000,0.000000,0.010000,0.010000,0.000000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000
3,3004,0.000000,0.000000,0.000000,0.060000,0.060000,0.050000,0.040000,0.020000,0.020000,0.030000,0.010000,0.020000,0.020000,0.010000,0.020000,0.010000,0.020000,0.010000,0.010000,0.020000,0.020000,0.010000,0.020000,0.010000,0.000000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3005,0.000000,0.000000,0.000000,0.000000,0.090000,0.070000,0.070000,0.040000,0.040000,0.020000,0.020000,0.030000,0.020000,0.020000,0.010000,0.010000,0.010000,0.010000,0.020000,0.010000,0.020000,0.010000,0.010000,0.020000,0.020000,0.010000,0.020000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
5,3006,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.060000,0.040000,0.020000,0.030000,0.020000,0.020000,0.020000,0.010000,0.040000,0.010000,0.020000,0.020000,0.010000,0.020000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.020000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
6,3007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.080000,0.040000,0.050000,0.020000,0.0